import package needed

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

download dataset(face celeba....) from website keggle


In [31]:
# !pip install kaggle 
# !pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission
!kaggle datasets download -d jessicali9530/celeba-dataset
# !kaggle datasets download -d gasgallo/faces-data-new
!unzip "/content/celeba-dataset.zip"


read dataset

In [3]:
import os
import PIL
import cv2
import glob
height=32
BATCH_SIZE=32
weight=32
channle=3
images=glob.glob("/content/img_align_celeba/img_align_celeba/*.jpg")
train_dataset=[]
i=0
for i in range(10000):
  img=PIL.Image.open(images[i]).resize((height,weight),PIL.Image.ANTIALIAS)
  train_dataset.append(np.array(img))

train_dataset=np.reshape(train_dataset,(10000,height,weight,channle)).astype("float32")
train_dataset=train_dataset/127.5-1.0

train_dataset=tf.data.Dataset.from_tensor_slices(train_dataset).shuffle(10000).batch(BATCH_SIZE)

In [12]:
def build_generator(seed_size,channles):
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(4*4*256,input_shape=(seed_size,)))
  model.add(tf.keras.layers.BatchNormalization(momentum=.8))
  model.add(tf.keras.layers.LeakyReLU(.2))


  model.add(tf.keras.layers.Reshape((4,4,256)))

  # model.add(tf.keras.layers.UpSampling2D())
  # model.add(tf.keras.layers.Conv2D(256,3,padding="same"))
  model.add(tf.keras.layers.Conv2DTranspose(256,3,strides=2,padding="same"))
  model.add(tf.keras.layers.BatchNormalization(momentum=.8))
  model.add(tf.keras.layers.LeakyReLU(.2))


  # model.add(tf.keras.layers.UpSampling2D())
  # model.add(tf.keras.layers.Conv2D(256,3,padding="same"))
  model.add(tf.keras.layers.Conv2DTranspose(128,3,strides=2,padding="same"))
  model.add(tf.keras.layers.BatchNormalization(momentum=.8))
  model.add(tf.keras.layers.LeakyReLU(.2))

  # model.add(tf.keras.layers.UpSampling2D())
  # model.add(tf.keras.layers.Conv2D(128,3,padding="same"))
  model.add(tf.keras.layers.Conv2DTranspose(64,3,strides=1,padding="same"))
  model.add(tf.keras.layers.BatchNormalization(momentum=.8))
  model.add(tf.keras.layers.LeakyReLU(.2))

  # model.add(tf.keras.layers.UpSampling2D((3,3)))
  model.add(tf.keras.layers.Conv2D(128,3,padding="same"))
  model.add(tf.keras.layers.Conv2DTranspose(32,3,strides=1,padding="same"))
  model.add(tf.keras.layers.BatchNormalization(momentum=.8))
  model.add(tf.keras.layers.LeakyReLU(.2))

  model.add(tf.keras.layers.Conv2DTranspose(16,3,strides=2,padding="same"))
  model.add(tf.keras.layers.BatchNormalization(momentum=.8))
  model.add(tf.keras.layers.LeakyReLU(.2))

  model.add(tf.keras.layers.Conv2D(3,3,padding="same",activation="tanh"))

  return model

In [14]:
def build_descreminator(image_shape):
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(32,3,strides=2,padding="same",input_shape=image_shape))
  model.add(tf.keras.layers.LeakyReLU(.2))

  model.add(tf.keras.layers.Dropout(.5))
  model.add(tf.keras.layers.Conv2D(64,3,strides=2,padding="same"))
  model.add(tf.keras.layers.BatchNormalization(momentum=.8))
  model.add(tf.keras.layers.LeakyReLU(.2))

  model.add(tf.keras.layers.Dropout(.5))
  model.add(tf.keras.layers.Conv2D(256,3,strides=2,padding="same"))
  model.add(tf.keras.layers.BatchNormalization(momentum=.8))
  model.add(tf.keras.layers.LeakyReLU(.2))


  model.add(tf.keras.layers.Dropout(.5))
  model.add(tf.keras.layers.Conv2D(512,3,strides=2,padding="same"))
  model.add(tf.keras.layers.BatchNormalization(momentum=.8))
  model.add(tf.keras.layers.LeakyReLU(.2))


  model.add(tf.keras.layers.Dropout(.5))
  model.add(tf.keras.layers.Conv2D(1024,3,strides=2,padding="same"))
  model.add(tf.keras.layers.BatchNormalization(momentum=.8))
  model.add(tf.keras.layers.LeakyReLU(.2))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1,activation="sigmoid"))

  return model

In [15]:
cross_ent=tf.keras.losses.BinaryCrossentropy(from_logits=1)
def discreminator_loss(real,fake):
  real_loss=cross_ent(tf.ones_like(real),real)
  fake_loss=cross_ent(tf.zeros_like(fake),fake)
  return real_loss+fake_loss

def generate_loss(fake):
  return cross_ent(tf.ones_like(fake),fake)

generate_optimizer=tf.keras.optimizers.Adam(1.5e-4,.5)
discreminator_optimizer=tf.keras.optimizers.Adam(1.5e-4,.5)
generator=build_generator(seed_size=100,channles=3)
discreminator=build_descreminator((height,weight,channle))

In [16]:
@tf.function
def train_step(images):
  seed=tf.random.normal([BATCH_SIZE,100])
  with tf.GradientTape() as gene_tape,tf.GradientTape() as discr_tape:
    generate_image=generator(seed,training=1)


    real=discreminator(images,training=1)
    fake=discreminator(generate_image,training=1)

    gene_loss=generate_loss(fake)
    discr_loss=discreminator_loss(real,fake)

    gradient_gene=gene_tape.gradient(gene_loss,generator.trainable_variables)
    gradient_disct=discr_tape.gradient(discr_loss,discreminator.trainable_variables)

    generate_optimizer.apply_gradients(zip(gradient_gene,generator.trainable_variables))
    discreminator_optimizer.apply_gradients(zip(gradient_disct,discreminator.trainable_variables))


    

In [17]:
PREVIEW_MARGIN=16
PREVIEW_ROWS=1
PREVIEW_COLS=1
GENERATE_SQUARE=32
SEED_SIZE=100
def save_images(cnt,noise):
  image_array = np.full(( 
      PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 
      PREVIEW_MARGIN + (PREVIEW_COLS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 3), 
      255, dtype=np.uint8)
  
  generated_images = generator.predict(noise)

  generated_images = 0.5 * generated_images + 0.5

  image_count = 0
  for row in range(PREVIEW_ROWS):
      for col in range(PREVIEW_COLS):
        r = row * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        c = col * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        image_array[r:r+GENERATE_SQUARE,c:c+GENERATE_SQUARE] \
            = generated_images[image_count] * 255
        image_count += 1

          
  output_path = os.path.join("/content",'output')
  if not os.path.exists(output_path):
    os.makedirs(output_path)
  
  filename = os.path.join(output_path,f"train-{cnt}.png")
  im =PIL.Image.fromarray(image_array)
  im.save(filename)



In [18]:
def train(dataset,epochs):
  fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, 
                                       SEED_SIZE))
  for epoch in range(epochs):
    gen_loss=[]
    dis_loss=[]
    for batch_image in dataset:
      train_step(batch_image)
    
    if epoch%2!=0:
      generator.save_weights("weights_generator_GAN.h5")
      discreminator.save_weights("weights_discreminator_GAN.h5")
      !cp weights_discreminator_GAN.h5 -r "/content/drive/My Drive"
      !cp weights_generator_GAN.h5 -r "/content/drive/My Drive"
      
    save_images(epoch,fixed_seed)
    print(epoch)

In [33]:
train(train_dataset,200)